# Recommendation System

In [2]:
#Libraries
import pandas as pd
import numpy as np
from random import *

In [3]:
#Load the dataset
data=pd.read_csv("facebook_combined.txt",sep=" ", header=None)

#Add column names
data.columns = ["node1", "node2"]

In [4]:
#Transform the graph to undirected
data2=pd.concat([data.node2,data.node1], axis=1)

#Rename the columns in order to merge the columns
data2.columns= ["node1", "node2"]
data=data.append(data2)

#Reset indexes
data = data.reset_index(drop=True)

In [5]:
#Create a sample graph dataset
test_data = pd.DataFrame([[5, 2], 
                       [9, 3],
                       [9, 11],
                       [3, 6],
                       [4, 6],
                       [5, 7],
                       [1, 11],
                       [6, 2],
                       [7, 9],
                       [8, 9],
                       [5, 11],
                       [6, 7],
                       [6, 11],
                       [7, 6],
                       [2, 11],
                       [11,2],
                       [2, 5],
                       [6, 2],
                       [2, 7],
                       [7, 2]],
                      columns=["node1", "node2"])

#### Recommending friends using Common neighbors (friend-of-friend (FoF) method)

In [6]:
##### Create the function for Common neighbors

def friendOfFriend(users, dataset, target):
    #Initialize
    l=list()
    friendships={}

    #Create friendships dict
    for node in users:
        #Create a list with the friends of node
        ls=dataset[dataset.node1 == node]['node2'].tolist()

        #Create a dictionary with key the node and value the list
        friendships[node]=ls

    # Initialize a dictionary with the intersections
    inter={}

    #Intersection between users
    for j in friendships:
        if (target != j) and (target not in friendships[j]) :
            intersection=(len(set(friendships.get(target)).intersection(set(friendships.get(j)))))
            #print(intersection)
            inter[j]=intersection
   
    #Create a sorted list, in ties we take the smallest ID
    lis=sorted(inter, key=inter.get, reverse=True)

    #Final Result
    return(lis[0:10]);
    

In [6]:
##### Test the code for the Sample
users=[1,2,3,4,5,6,7,8,9,10,11]
friendOfFriend(users, test_data, 5)

[6, 1, 7, 9, 11, 3, 4, 8, 10]

In [7]:
##### Test the code for the Original Dataset
users=list(range(0,4038))
print(friendOfFriend(users, data,107))
print(friendOfFriend(users, data,1126))
print(friendOfFriend(users, data,14))
print(friendOfFriend(users, data,35))

[513, 400, 559, 373, 492, 500, 378, 436, 431, 514]
[916, 1238, 1750, 1230, 1004, 1791, 1530, 1172, 1570, 1597]
[2, 17, 140, 111, 137, 162, 19, 333, 44, 243]
[46, 68, 99, 131, 175, 177, 225, 227, 278, 321]


###  Recommending friends using Jaccard coefficient

In [7]:
##### Create the function for Common neighbors using Jaccard coefficient

def JaccardCoefficient(users, dataset, target):
    #Initialize
    l=list()
    friendships={}

    #Create friendships dict
    for node in users:
        #Create a list with the friends of node
        ls=dataset[dataset.node1 == node]['node2'].tolist()

        #Create a dictionary with key the node and value the list
        friendships[node]=ls

    # Initialize a dictionary with the intersections
    inter={}

    #Intersection between users
    for j in friendships:
        if (target != j) and (target not in friendships[j]) :
            # Create union
            union=len(set(friendships.get(target)).union(set(friendships.get(j))))
            # Check for No zero denominator
            if (union != 0) :
                inter[j]=len(set(friendships.get(target)).intersection(set(friendships.get(j))))/union

    #Create a sorted list, in ties we take the smallest ID
    lis=sorted(inter, key=inter.get, reverse=True)

    #Final Result
    return(lis[0:10]);
    

In [9]:
##### Test the code for the Sample
users=[1,2,3,4,5,6,7,8,9,10,11]
JaccardCoefficient(users, test_data, 5)

[6, 1, 11, 9, 7, 3, 4, 8, 10]

In [10]:
##### Test the code for the Original Dataset
users=list(range(0,4038))

print(JaccardCoefficient(users, data,107))
print(JaccardCoefficient(users, data,1126))
print(JaccardCoefficient(users, data,14))
print(JaccardCoefficient(users, data,35))

[513, 400, 559, 492, 500, 373, 436, 378, 515, 514]
[916, 1750, 1230, 1530, 1004, 1238, 1172, 1791, 1789, 1597]
[2, 140, 17, 162, 111, 333, 44, 137, 19, 243]
[321, 11, 12, 15, 18, 37, 43, 74, 114, 209]


### Recommending friends using Adamic and Adar function

In [8]:
def AdamicAdarFunction(users, dataset, target):
    #Initialize
    l=list()
    friendships={}

    #Create friendships dict
    for node in users:
        #Create a list with the friends of node
        ls=dataset[dataset.node1 == node]['node2'].tolist()

        #Create a dictionary with key the node and value the list
        friendships[node]=ls

    # Initialize a dictionary with the intersections
    inter={}

    #Intersection between users
    for j in friendships:
        if (target != j) and (target not in friendships[j]) :
            intersection = set(friendships.get(target)).intersection(set(friendships.get(j)))

            # Adamic and Adar score calculation
            sum = 0
            for k in intersection :
                if ( k in friendships.keys()) and (friendships[k] != []):
                    sum = sum+np.log(len(friendships[k]))

            if (sum != 0) :        
                inter[j]=1/sum
            else:
                inter[j]=0
            
    #Create a sorted list, in ties we take the smallest ID
    lis=sorted(inter, key=inter.get, reverse=True)

    #Final Result
    return(lis[0:10]);

In [12]:
##### Test the code for the Sample
users=[1,2,3,4,5,6,7,8,9,10,11]
AdamicAdarFunction(users, test_data, 5)

[7, 11, 6, 1, 3, 4, 8, 9, 10]

In [13]:
##### Test the code for the Original Dataset
users=list(range(0,4038))

print(AdamicAdarFunction(users, data,107))
print(AdamicAdarFunction(users, data,1126))
print(AdamicAdarFunction(users, data,14))
print(AdamicAdarFunction(users, data,35))

[862, 3437, 3440, 3456, 3495, 3501, 3525, 3540, 3550, 3556]
[136, 1912, 1916, 1920, 1926, 1932, 1940, 1945, 1947, 1948]
[1, 3, 4, 5, 6, 7, 8, 9, 10, 11]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


### Evaluation of the recommendation system

In [9]:
# Create users list
users=list(range(100,4100,100))
data100=data[(data.node2%100 == 0) & (data.node1%100 == 0) & (data.node1 != 0) & (data.node2 != 0)]

#Run the functions
fofList=friendOfFriend(users, data100,100)
JaccardList=JaccardCoefficient(users, data100,100)
AdamicAdarList=AdamicAdarFunction(users, data100,100)

#Similarity Percentage of FoF and Jaccard
s1=len(set(fofList).intersection(set(JaccardList)))*10

#Similarity Percentage of FoF and Adamic and Adar
s2=len(set(fofList).intersection(set(AdamicAdarList)))*10

#Similarity Percentage of Jaccard and Adamic and Adar
s3=len(set(AdamicAdarList).intersection(set(JaccardList)))*10

#Average Similarity (%)
print("The average similarity is:",round((s1+s2+s3)/3,2),"%")

The average similarity is: 46.67 %


### Forecast Recommendations

In [10]:
#Choose a pair F1 AND F2

def evaluationFunction(dataset,users,F1,F2):

    ####Remove the relationship
    
    #First we find the connection F1-F2
    l1=dataset[dataset.node2 == F1].index
    l2=dataset[dataset.node1 == F2 ].index
    rm1=set(l1).intersection(set(l2))
    
    #Then we find the connection F2-F1
    l1=dataset[dataset.node2 == F2 ].index
    l2=dataset[dataset.node1 == F1].index
    rm2=set(l1).intersection(set(l2))

    #We create the union
    rm=rm1.union(rm2)
    if rm == set():
        return(None);
    
    #Remove the elements of the set rm
    for i in rm:
        dataset=dataset.drop(i)
    
    ###FoF (friend-of-friend)
    if ((F1 in friendOfFriend(users, dataset,F2)) and (F2 in friendOfFriend(users, dataset,F1))): 
        
        #Compute the recommendations for F1
        Friend1=10 - friendOfFriend(users, dataset,F1).index(F2)

        #Compute the resommentdations for F2
        Friend2=10 - friendOfFriend(users, dataset,F2).index(F1)


        ####Compute the score
        scoreFoF=(Friend1+Friend2)/2
        
    else:
        scoreFoF= 0
    
    ###Jaccard
    if ((F1 in JaccardCoefficient(users, dataset,F2)) and (F2 in JaccardCoefficient(users, dataset,F1))): 
        
        #Compute the recommendations for F1
        Friend1=10 - JaccardCoefficient(users, dataset,F1).index(F2)

        #Compute the resommentdations for F2
        Friend2=10 - JaccardCoefficient(users, dataset,F2).index(F1)

        ####Check if either of these does not exist

        ####Compute the score
        scoreJaccard=(Friend1+Friend2)/2
        
    else:
        scoreJaccard = 0
    
    ###AdamicAdar
    if ((F1 in AdamicAdarFunction(users, dataset,F2)) and (F2 in AdamicAdarFunction(users, dataset,F1))):
        
        #Compute the recommendations for F1
        Friend1=10 - AdamicAdarFunction(users, dataset,F1).index(F2)

        #Compute the resommentdations for F2
        Friend2=10 - AdamicAdarFunction(users, dataset,F2).index(F1)

        ####Check if either of these does not exist

        ####Compute the score
        scoreAdamicAdar=(Friend1+Friend2)/2
        
    else:
        scoreAdamicAdar = 0       
    
    return(scoreFoF,scoreJaccard,scoreAdamicAdar);

In [14]:
#Function for iterations
def finalScore(dataset,users, n):
    eval_scores=[]
    for l in list(range(0,n)):
        
        friends=sample(users,2) #take 2 random users
        evaluationOutput=evaluationFunction(dataset,users,friends[0],friends[1])
        
        if (evaluationOutput!= None):
            eval_scores.append(evaluationFunction(dataset,users,friends[0],friends[1]))
        
    #Final Print
    if eval_scores != [] :
        #Calculate the average value
        scores=np.mean(eval_scores,axis=0)

        
    #Final Print
    if eval_scores != [] :
        #Calculate the average value
        scores=np.mean(eval_scores,axis=0)
        
        print("FoF score is: ", round(scores[0],3),"/10")
        print("Jaccard score is: ", round(scores[1],3),"/10")
        print("Adamic Adar score is: ", round(scores[2],3),"/10")
        
        
    return;       

In [15]:
#Create the scores for the test_data (for 100 iterations)
users=[1,2,3,4,5,6,7,8,9,10,11]
finalScore(test_data,users,100)

FoF score is:  6.152 /10
Jaccard score is:  5.87 /10
Adamic Adar score is:  6.522 /10


In [16]:
users=list(range(0,4038))
finalScore(data,users,100)

FoF score is:  4.25 /10
Jaccard score is:  4.5 /10
Adamic Adar score is:  0.0 /10
